In [1]:
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import dataset
import test_lstm

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2023-02-27 22:50:46.584551: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries

In [3]:
# # no split by person
X, Y = dataset.LSTM_preprocessing(truth_trial_path, lie_trial_path)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [4]:
# split by person
# xTrain, yTrain, xTest, yTest = dataset.createDatasetLSTM(lie_trial_path, truth_trial_path, 0.2, byPerson=False)

In [9]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=False)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))

  model.add(Dropout(drop))
  model.add(Dense(1, activation='sigmoid')) #softmax

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 32)                6016      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 6,049
Trainable params: 6,049
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1536/1536 [==============================] - 8s 4ms/step - loss: 0.5697 - accuracy: 0.7310 - val_loss: 0.5090 - val_accuracy: 0.7873
Epoch 2/10
1536/1536 [==============================] - 5s 3ms/step - loss: 0.4878 - accuracy: 0.7878 - val_loss: 0.4642 - val_accuracy: 0.7910
Epoch 3/10
1536/1536 [======

In [6]:
prediction = MODEL.predict(xTest[0:250])
label = yTest[0:250]

for i in range(len(prediction)):
  if prediction[i] > 0.5:
    prediction[i] = 1
  else:
    prediction[i] = 0

accuracy = 0
for i in range(len(label)):
  if prediction[i] == label[i]:
    accuracy += 1
  
print("Accuracy: ", accuracy/len(label))

8/8 [==============================] - 0s 2ms/step
Accuracy:  0.84


In [19]:
import importlib
importlib.reload(test_lstm)

lie_test_path = './test/Truth/processed/'

prediction_test = test_lstm.perdictSingleVideo(lie_test_path, MODEL)
print(prediction_test)


for i in range(len(prediction_test)):
  if prediction_test[i] > 0.5:
    prediction_test[i] = 1
  else:
    prediction_test[i] = 0

accuracy = 0
for i in range(len(prediction_test)):
    if prediction_test[i] == 1:
        accuracy += 1

print("Accuracy: ", accuracy/len(prediction_test))

(520, 10, 14)
17/17 [==============================] - 0s 3ms/step
[[0.00440359]
 [0.00438195]
 [0.00435586]
 [0.00433787]
 [0.00431839]
 [0.00429675]
 [0.00427732]
 [0.00419091]
 [0.00419339]
 [0.00418965]
 [0.00418459]
 [0.00417796]
 [0.00410526]
 [0.00410525]
 [0.00410524]
 [0.00410522]
 [0.00410515]
 [0.00410506]
 [0.00410486]
 [0.00410453]
 [0.00410397]
 [0.00410318]
 [0.00410201]
 [0.00410019]
 [0.00409803]
 [0.00409399]
 [0.00408985]
 [0.00408519]
 [0.00407866]
 [0.00407203]
 [0.00406465]
 [0.00405872]
 [0.00405191]
 [0.00404277]
 [0.00403493]
 [0.00400781]
 [0.00400215]
 [0.00400267]
 [0.00400587]
 [0.00401291]
 [0.00402496]
 [0.00403972]
 [0.00405382]
 [0.00406549]
 [0.00407365]
 [0.00408086]
 [0.00408585]
 [0.00409065]
 [0.00409422]
 [0.00409712]
 [0.0040991 ]
 [0.00410065]
 [0.00410167]
 [0.00410258]
 [0.00410317]
 [0.00410356]
 [0.00410389]
 [0.00410419]
 [0.00410443]
 [0.00410466]
 [0.00410484]
 [0.00410505]
 [0.00410516]
 [0.00410529]
 [0.00410545]
 [0.0041056 ]
 [0.00410